<a href="https://colab.research.google.com/github/bekku/deeplearning/blob/master/%E8%92%B8%E7%95%99%E6%9C%80%E9%81%A9%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E9%81%BA%E4%BC%9D%E7%9A%84%E6%8E%A2%E7%B4%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import time

from google.colab import drive #インポート
drive.mount('/content/gdrive') #GoogleDriveのマウント

torch.cuda.is_available()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


True

In [ ]:
!pip install deap

     |████████████████████████████████| 163kB 2.7MB/s 


In [ ]:
def load_cifar10(batch=128):
    train_loader = DataLoader(
        datasets.CIFAR10('./data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                [0.5, 0.5, 0.5],  # RGB 平均
                                [0.5, 0.5, 0.5]   # RGB 標準偏差
                                )
                         ])),
        batch_size=batch,
        shuffle=True
    )

    test_loader = DataLoader(
        datasets.CIFAR10('./data',
                         train=False,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                 [0.5, 0.5, 0.5],  # RGB 平均
                                 [0.5, 0.5, 0.5]  # RGB 標準偏差
                             )
                         ])),
        batch_size=batch,
    )

    return {'train': train_loader, 'test': test_loader}

# **教師モデルの設定**

In [ ]:
#教師モデル
#####ドロップアウトを除いた推論用モデル#######
#ここで温度を変更して、softターゲットとする。


class T_CNN(torch.nn.Module):
    def __init__(self):
        super(T_CNN, self).__init__()

        self.conv1 = torch.nn.Conv2d(3, 32, 3, 1, 0)
        self.conv2 = torch.nn.Conv2d(32, 32, 3)
        self.conv3 = torch.nn.Conv2d(32, 64, 3)
        self.conv4 = torch.nn.Conv2d(64, 64, 3)
        
        self.dropout1 = torch.nn.Dropout2d(p=0.25)
        self.dropout2 = torch.nn.Dropout2d(p=0.5)

        self.pool = torch.nn.MaxPool2d(2, 2)

        self.fc1 = torch.nn.Linear(64 * 5 * 5, 512) 
        self.fc2 = torch.nn.Linear(512, 10)

    def forward(self, x, T):
        x = f.relu(self.conv1(x))
        x = f.relu(self.conv2(x))
        x = self.pool(x)
        
        x = f.relu(self.conv3(x))
        x = f.relu(self.conv4(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 5 * 5)
        x = f.relu(self.fc1(x))
        x = self.fc2(x)
        x = x/T

        return f.softmax(x)

class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = torch.nn.Linear(32 * 32 * 3, 1024)
        self.fc2 = torch.nn.Linear(1024, 10)

    def forward(self, x, T):
        x = f.relu(self.fc1(x))
        x = self.fc2(x)
        x = x/T
        return f.log_softmax(x)
# 教師モデルと生徒モデルのcross_entropyの損失関数の定義
class T_S_Closs_entropy(torch.nn.Module):
    def __init__(self):
        super(T_S_Closs_entropy, self).__init__()

    def forward(self, p, q):
        #pが教師モデル、qが生徒モデルの出力値(既に対数化してます)です。
        # c = torch.max(q)
        loss = p*q
        loss = torch.sum(loss)
        loss*=-1
        return loss

Tnet: T_CNN = T_CNN()
param = torch.load('/content/gdrive/My Drive/コード/teacher_model_weight.pth')
Tnet.load_state_dict(param)
loader = load_cifar10()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
Tnet.to(device)
s_max=0
for i in range(10):
    t1 = time.time()
    correct = 0
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = Tnet(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()
    s_max += t2-t1
a_max = float(correct / 10000)
print(a_max)
t2 = time.time()
s_max /= 10
print(f"経過時間：{s_max}")

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.7364
経過時間：3.0206634283065794


# **評価関数 evaluater の定義**


In [ ]:
# 温度付きモデル生成関数
class CreateNet(torch.nn.Module):
    def __init__(self, NN_model_list):
        super(CreateNet, self).__init__()

        self.NN_model_list_2=[]
        for i in range(len(NN_model_list)):
            if NN_model_list[i]!=0:
                self.NN_model_list_2.append(NN_model_list[i])

        NN_model_list=[len(self.NN_model_list_2)]+self.NN_model_list_2

        # 隠れ層0の時エラー処理用 if。
        if NN_model_list[0] > 0:
            for i in range(len(NN_model_list)-1):
                if i==0:
                   self._modules["fc"+str(i+1)]= torch.nn.Linear(3 * 32 * 32, NN_model_list[1])
                   #入力層
                else:
                    self._modules["fc"+str(i+1)]= torch.nn.Linear(NN_model_list[i], NN_model_list[i+1])
                   #出力層 fcリストの個数は、常にNN_model_list[0]+1である。
            self._modules["fc"+str(i+2)]= torch.nn.Linear(NN_model_list[-1], 10)

        else:
            self._modules["fc"+str(1)]= torch.nn.Linear(3 * 32 * 32, 10)

    def forward(self, x, T):
        for i in range(len(self._modules)-1):
            x = f.relu(self._modules["fc"+str(i+1)](x))
        x = self._modules["fc"+str(len(self._modules))](x)
        x = x/T
        return f.log_softmax(x)

# 速度と精度の評価関数、これが最大化されたい。
def speed_acc_normalization(spe_max,spe_min,spe,acc_max,acc_min,acc):
    if spe_min<=spe<=spe_max:
        speed_normalization = ((spe_max-spe+1) - spe_min) / ( (spe_max - spe_min) * 2 )
    else:
        speed_normalization=0
    if acc_min<=acc<=acc_max:
      accuracy_normalization = (acc - acc_min) / ( (acc_max - acc_min) * 2 )
    else:
        accuracy_normalization=0
    print("----------------")
    print("速度:{} ,精度:{}".format(spe,acc))
    print("速度評価値:{} ,精度評価値:{}".format(speed_normalization,accuracy_normalization))
    print("評価指標:{}".format(speed_normalization + accuracy_normalization))
    print("----------------")

    return speed_normalization + accuracy_normalization


loaders = load_cifar10()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
Tnet: T_CNN = T_CNN()
param = torch.load('/content/gdrive/My Drive/コード/teacher_model_weight.pth')
Tnet.load_state_dict(param)
Tnet.to(device)
criterion = T_S_Closs_entropy()

#　遺伝的評価関数、引数 : gane=層ごとの2の条数-リスト、temp = 温度、スピード最大、精度最大。
def evaluater(gane):
    # 入力された乗数から、本当のモデルのノード数とする。
    model_gane=[]
    for i in gane:
      if i==0:
          model_gane.append(0)
      else:
          model_gane.append(2**i)

    #温度設定
    temp=20

    # 学習回数
    epoch = 30
    net: torch.nn.Module = CreateNet(model_gane)
    net.to(device)

    optimizer = torch.optim.Adam(params=net.parameters(), lr=0.001)

    for e in range(1,epoch+1):
        """　学習　開始"""
        loss = None
        net.train()
        for data, target in loaders['train']:
                
            data = data.to(device)
            target = target.to(device)

            max_model_outputs = Tnet(data,temp)
            data = data.view(-1, 32 * 32 * 3)
            output = net(data,temp)

            optimizer.zero_grad()
            loss = criterion(max_model_outputs,output)

            loss.backward()
            optimizer.step()
        if e%10==0:
            print('temp: {} ,Training log: {} ,Loss: {}'.format(temp,e,loss.item()))

    """  学習したモデルの速度と精度の評価をする。(推論)"""
    net.eval()
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loaders['test']:
            images = images.to(device) 
            labels = labels.to(device)
            data = images.view(-1, 32 * 32 * 3)
            #　蒸留モデルは、推論時は温度を1に戻します。
            outputs = net(data,1)
            _, predicted = torch.max(outputs.data, 1)
            #torch.max(a, axis) (axis=0:col, axis=1:row)ごとに最大値を取ってくれている。
            correct += (predicted == labels).sum().item()

    acc = float(correct / 10000)
    t2 = time.time()
    elapsed_time = t2-t1

    # accとelapsedは、教師モデルの出力を受け取るようにしておきましょう。
    return (speed_acc_normalization(s_max,0,elapsed_time,a_max,0,acc),)
print("出力")

Files already downloaded and verified
Files already downloaded and verified
出力


In [ ]:
print(evaluater([0,0,10]))
print(evaluater([0]))
from deap import base, creator, tools
from deap.benchmarks import ackley
print(ackley([5,5]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


temp: 10 ,Training log: 10 ,Loss: 179.6783447265625
----------------
速度:2.5068113803863525 ,精度:0.5331
速度評価値:0.2640374041051057 ,精度評価値:0.36196360673546984
評価指標:0.6260010108405756
----------------
(0.6260010108405756,)
temp: 10 ,Training log: 10 ,Loss: 181.74058532714844
----------------
速度:2.55118465423584 ,精度:0.3774
速度評価値:0.2570886691657695 ,精度評価値:0.2562466051059207
評価指標:0.5133352742716901
----------------
(0.5133352742716901,)
(12.642411176571153,)


In [ ]:
import random
from deap import base, creator, tools


# 適応度クラスの作成
# 最大化したい場合1.0
# 最小化したい場合-1.0
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# 個体クラスの作成
creator.create("Individual", list, fitness=creator.FitnessMax)

# Toolboxの作成
toolbox = base.Toolbox()

# 遺伝子を生成する関数"attr_gene"を登録
# 0~10000の値をランダムに生成する。
toolbox.register("attr_gene", random.randrange, 0, 14)

# 個体を生成する関数”individual"を登録
# initRepeat(creator.Individual,遺伝子生成関数, N)
# N回遺伝子生成関数を実行し、creator.Individualに格納して返す関数
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_gene, 8)

# 世代(個体集団)を生成する関数"population"を登録
# 個体生成したものを集めて、世代化
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 評価関数"evaluate"を登録
toolbox.register("evaluate", evaluater)

# 交叉を行う関数"mate"を登録
toolbox.register("mate", tools.cxTwoPoint)

# 変異を行う関数"mutate"を登録
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
# toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
# toolbox.register("mutate", tools.mutGaussian, mu=[
#                  0.0, 0.0], sigma=[200.0, 200.0], indpb=0.2)

# 個体選択法"select"を登録
toolbox.register("select", tools.selTournament, tournsize=3)


def main():

    random.seed(1)

    # GAパラメータ
    N_GEN = 20      # 繰り返し世代数
    POP_SIZE = 10  # 集団内の個体数
    CX_PB = 0.5      # 交叉確率
    MUT_PB = 0.2     # 変異確率

    # 個体集団の生成
    pop = toolbox.population(n=POP_SIZE)
    print("遺伝的探索開始")
    print(pop)

    # 個体集団の適応度の評価
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    print("  Evaluated %i individuals" % len(pop))
    # 適応度の抽出
    fits = [ind.fitness.values[0] for ind in pop]

    print("-----------------------")
    # 進化ループ開始
    g = 0
    while g < N_GEN:

        g = g + 1
        print("-- Generation %i --" % g)

        # 次世代個体の選択・複製
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))
        print(0,offspring)

        # 交叉
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # 交叉させる個体を選択
            if random.random() < CX_PB:
                toolbox.mate(child1, child2)

                # 交叉させた個体は適応度を削除する
                del child1.fitness.values
                del child2.fitness.values

        # 変異
        for mutant in offspring:

            # 変異させる個体を選択
            if random.random() < MUT_PB:
                toolbox.mutate(mutant)

                # 変異させた個体は適応度を削除する
                del mutant.fitness.values

        # 適応度を削除した個体について適応度の再評価を行う
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        print(invalid_ind)
        fitnesses = map(toolbox.evaluate, invalid_ind)

        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # 個体集団を新世代個体集団で更新
        pop[:] = offspring

        # 新世代の全個体の適応度の抽出
        fits = [ind.fitness.values[0] for ind in pop]

        # 適応度の統計情報の出力
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)


    print("-- End of (successful) evolution --")

    # 最良個体の抽出
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


if __name__ == '__main__':

    main()

遺伝的探索開始
[[2, 9, 13, 12, 12, 1, 4, 1], [7, 12, 7, 7, 10, 6, 12, 3], [1, 7, 0, 13, 6, 6, 9, 12], [12, 0, 11, 7, 4, 11, 12, 3], [9, 1, 5, 0, 0, 0, 10, 8], [0, 6, 10, 3, 6, 11, 0, 8], [3, 12, 7, 7, 8, 3, 5, 3], [10, 3, 12, 7, 4, 0, 6, 13], [8, 10, 1, 2, 10, 11, 13, 4], [1, 11, 5, 11, 11, 8, 6, 8]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


KeyboardInterrupt: ignored

In [ ]:
import random
from deap import base, creator, tools


# 適応度クラスの作成
# 最大化したい場合1.0
# 最小化したい場合-1.0
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# 個体クラスの作成
creator.create("Individual", list, fitness=creator.FitnessMax)

# Toolboxの作成
toolbox = base.Toolbox()

# 遺伝子を生成する関数"attr_gene"を登録
# 0~10000の値をランダムに生成する。
toolbox.register("attr_gene", random.randrange, 0, 14)

# 個体を生成する関数”individual"を登録
# initRepeat(creator.Individual,遺伝子生成関数, N)
# N回遺伝子生成関数を実行し、creator.Individualに格納して返す関数
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_gene, 5)

# 世代(個体集団)を生成する関数"population"を登録
# 個体生成したものを集めて、世代化
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 評価関数"evaluate"を登録
toolbox.register("evaluate", evaluater)

# 交叉を行う関数"mate"を登録
toolbox.register("mate", tools.cxTwoPoint)

# 変異を行う関数"mutate"を登録
toolbox.register("mutate", tools.mutUniformInt, low=0, up=14, indpb=0.2)

# 個体選択法"select"を登録
toolbox.register("select", tools.selTournament, tournsize=3)


def main():

    random.seed()

    # GAパラメータ
    N_GEN = 20      # 繰り返し世代数
    POP_SIZE = 10   # 集団内の個体数
    CX_PB = 0.5     # 交叉確率
    MUT_PB = 0.05    # 変異確率0.05

    # 個体集団の生成
    pop = toolbox.population(n=POP_SIZE)
    print("遺伝的探索開始")
    pop[-1]=[10,0,0,0,0]
    print(pop)

    # 個体集団の適応度の評価
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    print("  Evaluated %i individuals" % len(pop))
    # 適応度の抽出
    fits = [ind.fitness.values[0] for ind in pop]

    print("-----------------------")
    # 進化ループ開始
    g = 0
    while g < N_GEN:

        g = g + 1
        print("-- Generation %i --" % g)

        # 次世代個体の選択・複製
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))
        print(offspring)

        # 交叉
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # 交叉させる個体を選択
            if random.random() < CX_PB:
                toolbox.mate(child1, child2)

                # 交叉させた個体は適応度を削除する
                del child1.fitness.values
                del child2.fitness.values

        # 変異
        for mutant in offspring:

            # 変異させる個体を選択
            if random.random() < MUT_PB:
                print("変異前:{}".format(mutant))
                toolbox.mutate(mutant)
                print("変異後:{}".format(mutant))

                # 変異させた個体は適応度を削除する
                del mutant.fitness.values


        # 適応度を削除した個体について適応度の再評価を行う
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        print(invalid_ind)
        fitnesses = map(toolbox.evaluate, invalid_ind)

        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # 個体集団を新世代個体集団で更新
        pop[:] = offspring

        # 新世代の全個体の適応度の抽出
        fits = [ind.fitness.values[0] for ind in pop]

        # 適応度の統計情報の出力
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)


    print("-- End of (successful) evolution --")

    # 最良個体の抽出
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


if __name__ == '__main__':

    main()

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


遺伝的探索開始
[[7, 4, 12, 10, 8], [13, 2, 8, 7, 7], [2, 7, 4, 2, 11], [5, 11, 3, 6, 12], [2, 13, 1, 9, 1], [11, 6, 9, 2, 3], [8, 12, 2, 1, 4], [7, 10, 10, 13, 10], [10, 1, 3, 3, 13], [10, 0, 0, 0, 0]]
temp: 20 ,Training log: 10 ,Loss: 183.13400268554688
temp: 20 ,Training log: 20 ,Loss: 182.81861877441406
----------------
速度:2.535641670227051 ,精度:0.5212
速度評価値:0.23740016793967664 ,精度評価値:0.35388375882672457
評価指標:0.5912839267664012
----------------
temp: 20 ,Training log: 10 ,Loss: 182.9959716796875
temp: 20 ,Training log: 20 ,Loss: 183.17770385742188
----------------
速度:2.8575236797332764 ,精度:0.4919
速度評価値:0.18235727396358603 ,精度評価値:0.3339896795219989
評価指標:0.5163469534855849
----------------
temp: 20 ,Training log: 10 ,Loss: 183.62460327148438
temp: 20 ,Training log: 20 ,Loss: 183.29318237304688
----------------
速度:2.4625673294067383 ,精度:0.3185
速度評価値:0.24989612321321103 ,精度評価値:0.216254752851711
評価指標:0.466150876064922
----------------
temp: 20 ,Training log: 10 ,Loss: 183.0638427734375
temp: 20 

# **最適モデルの評価確認**

In [ ]:
def evaluater(gane):
    # 入力された乗数から、本当のモデルのノード数とする。
    model_gane=[]
    for i in gane:
      if i==0:
          model_gane.append(0)
      else:
          model_gane.append(2**i)

    #温度設定
    temp=10

    # 学習回数
    epoch = 50
    net: torch.nn.Module = CreateNet(model_gane)
    net.to(device)

    optimizer = torch.optim.Adam(params=net.parameters(), lr=0.001)

    for e in range(1,epoch+1):
        """　学習　開始"""
        loss = None
        net.train()
        for data, target in loaders['train']:
                
            data = data.to(device)
            target = target.to(device)

            max_model_outputs = Tnet(data,temp)
            data = data.view(-1, 32 * 32 * 3)
            output = net(data,temp)

            optimizer.zero_grad()
            loss = criterion(max_model_outputs,output)

            loss.backward()
            optimizer.step()
        if e%10==0:
            print('temp: {} ,Training log: {} ,Loss: {}'.format(temp,e,loss.item()))

    torch.save(net.state_dict(), 'GA_student_NN_12_8_11_3_6_11.pth')

    """  学習したモデルの速度と精度の評価をする。(推論)"""
    net.eval()
    elapsed_time=0
    for i in range(10):
        correct = 0
        t1 = time.time()
        with torch.no_grad():
            for images, labels in loaders['test']:
                images = images.to(device) 
                labels = labels.to(device)
                data = images.view(-1, 32 * 32 * 3)
                #　蒸留モデルは、推論時は温度を1に戻します。
                outputs = net(data,1)
                _, predicted = torch.max(outputs.data, 1)
                #torch.max(a, axis) (axis=0:col, axis=1:row)ごとに最大値を取ってくれている。
                correct += (predicted == labels).sum().item()

        acc = float(correct / 10000)
        t2 = time.time()
        elapsed_time += t2-t1
    elapsed_time /=10

    # accとelapsedは、教師モデルの出力を受け取るようにしておきましょう。
    return speed_acc_normalization(s_max,0,elapsed_time,a_max,0,acc)
print("---最適モデル---")
print(evaluater([10,8]))


---最適モデル---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


temp: 10 ,Training log: 10 ,Loss: 179.47813415527344
temp: 10 ,Training log: 20 ,Loss: 178.94952392578125
temp: 10 ,Training log: 30 ,Loss: 179.17417907714844
temp: 10 ,Training log: 40 ,Loss: 178.1798858642578
temp: 10 ,Training log: 50 ,Loss: 177.97914123535156
----------------
速度:2.7107788801193236 ,精度:0.583
速度評価値:0.2527039161180319 ,精度評価値:0.39584464964693095
評価指標:0.6485485657649628
----------------
0.6485485657649628


# **1層モデルの評価確認（基準として）**

In [ ]:
def evaluater(gane):
    # 入力された乗数から、本当のモデルのノード数とする。
    model_gane=[]
    for i in gane:
      if i==0:
          model_gane.append(0)
      else:
          model_gane.append(2**i)

    #温度設定
    temp=10

    # 学習回数
    epoch = 50
    net: torch.nn.Module = CreateNet(model_gane)
    net.to(device)

    optimizer = torch.optim.Adam(params=net.parameters(), lr=0.001)

    for e in range(1,epoch+1):
        """　学習　開始"""
        loss = None
        net.train()
        for data, target in loaders['train']:
                
            data = data.to(device)
            target = target.to(device)

            max_model_outputs = Tnet(data,temp)
            data = data.view(-1, 32 * 32 * 3)
            output = net(data,temp)

            optimizer.zero_grad()
            loss = criterion(max_model_outputs,output)

            loss.backward()
            optimizer.step()
        if e%10==0:
            print('temp: {} ,Training log: {} ,Loss: {}'.format(temp,e,loss.item()))

    """  学習したモデルの速度と精度の評価をする。(推論)"""
    net.eval()
    elapsed_time=0
    for i in range(10):
        correct = 0
        t1 = time.time()
        with torch.no_grad():
            for images, labels in loaders['test']:
                images = images.to(device) 
                labels = labels.to(device)
                data = images.view(-1, 32 * 32 * 3)
                #　蒸留モデルは、推論時は温度を1に戻します。
                outputs = net(data,1)
                _, predicted = torch.max(outputs.data, 1)
                #torch.max(a, axis) (axis=0:col, axis=1:row)ごとに最大値を取ってくれている。
                correct += (predicted == labels).sum().item()

        acc = float(correct / 10000)
        t2 = time.time()
        elapsed_time += t2-t1
    elapsed_time /=10

    # accとelapsedは、教師モデルの出力を受け取るようにしておきましょう。
    return speed_acc_normalization(s_max,0,elapsed_time,a_max,0,acc)
print("---1024の1層モデル---")
print(evaluater([10]))


---1024の1層モデル---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


temp: 10 ,Training log: 10 ,Loss: 180.44435119628906
temp: 10 ,Training log: 20 ,Loss: 178.97694396972656
temp: 10 ,Training log: 30 ,Loss: 178.2796630859375
temp: 10 ,Training log: 40 ,Loss: 178.634765625
temp: 10 ,Training log: 50 ,Loss: 178.77899169921875
----------------
速度:2.733385610580444 ,精度:0.5568
速度評価値:0.24943607362980783 ,精度評価値:0.3780554046713742
評価指標:0.627491478301182
----------------
0.627491478301182
